In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer
from transformers import RobertaModel

In [2]:
from datasets import load_dataset

dataset = load_dataset("tweet_eval", "emoji")

Using the latest cached version of the module from C:\Users\User\.cache\huggingface\modules\datasets_modules\datasets\tweet_eval\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343 (last modified on Thu Apr  6 10:44:13 2023) since it couldn't be found locally at tweet_eval., or remotely on the Hugging Face Hub.
Found cached dataset tweet_eval (C:/Users/User/.cache/huggingface/datasets/tweet_eval/emoji/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 45000
})

In [4]:
dataset["validation"]

Dataset({
    features: ['text', 'label'],
    num_rows: 5000
})

In [5]:
dataset["validation"]

Dataset({
    features: ['text', 'label'],
    num_rows: 5000
})

In [7]:
# Load a pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")


In [8]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

In [9]:
# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Get the input_ids from the tokenized dataset for train, validation, and test sets
train_input_ids = [example['input_ids'] for example in tokenized_dataset['train']]
val_input_ids = [example['input_ids'] for example in tokenized_dataset['validation']]
test_input_ids = [example['input_ids'] for example in tokenized_dataset['test']]

# Convert input_ids to PyTorch tensors
train_input_ids = torch.tensor(train_input_ids, dtype=torch.long)
val_input_ids = torch.tensor(val_input_ids, dtype=torch.long)
test_input_ids = torch.tensor(test_input_ids, dtype=torch.long)

# Concatenate the input_ids tensors along the first dimension
input_ids = torch.cat((train_input_ids, val_input_ids, test_input_ids), dim=0)


Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\tweet_eval\emoji\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-bd0aaef3bae969b0.arrow


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\tweet_eval\emoji\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-db007c17860440dc.arrow


In [10]:
# Tokenize the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Get the attention_masks from the tokenized dataset for train, validation, and test sets
train_attention_mask = [example['attention_mask'] for example in tokenized_dataset['train']]
val_attention_mask = [example['attention_mask'] for example in tokenized_dataset['validation']]
test_attention_mask = [example['attention_mask'] for example in tokenized_dataset['test']]

# Convert attention_mask to PyTorch tensors
train_attention_mask = torch.tensor(train_attention_mask, dtype=torch.long)
val_attention_mask = torch.tensor(val_attention_mask, dtype=torch.long)
test_attention_mask = torch.tensor(test_attention_mask, dtype=torch.long)

# Concatenate the input_ids tensors along the first dimension
attention_masks = torch.cat((train_attention_mask, val_attention_mask, test_attention_mask), dim=0)


Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\tweet_eval\emoji\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-c9019aa66d29999a.arrow
Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\tweet_eval\emoji\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-52b73ff83e38f868.arrow
Loading cached processed dataset at C:\Users\User\.cache\huggingface\datasets\tweet_eval\emoji\1.1.0\12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343\cache-db007c17860440dc.arrow


In [11]:
print(input_ids)

tensor([[    0, 21583,  1390,  ...,     1,     1,     1],
        [    0, 14699,    13,  ...,     1,     1,     1],
        [    0, 16587,   657,  ...,     1,     1,     1],
        ...,
        [    0,   510,  5090,  ...,     1,     1,     1],
        [    0,   170,   657,  ...,     1,     1,     1],
        [    0, 38065,  2838,  ...,     1,     1,     1]])


In [12]:
print(attention_masks)

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [13]:
# Split the dataset into train, validation, and test sets
train_dataset = tokenized_dataset["train"]
val_dataset = tokenized_dataset["validation"]
test_dataset = tokenized_dataset["test"]

In [14]:
# Convert the datasets to PyTorch tensors
train_dataset = train_dataset.remove_columns(["text"])
val_dataset = val_dataset.remove_columns(["text"])
test_dataset = test_dataset.remove_columns(["text"])

In [15]:
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [16]:


class EmojiDetectionModel(nn.Module):
    def __init__(self, num_classes=20):
        super(EmojiDetectionModel, self).__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.roberta.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [18]:
# instantiate your model
emoji_model = EmojiDetectionModel(num_classes=20).to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
# define your loss function
criterion = nn.CrossEntropyLoss()

# define your optimizer
optimizer = torch.optim.Adam(emoji_model.parameters(), lr=0.001)

In [20]:
# set up your training, validation, and test dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [21]:
for batch in train_loader:
    print(batch)


{'label': tensor([ 1,  2, 18,  1,  2,  6, 17,  1, 17,  3,  6,  2,  2,  0,  0,  4]), 'input_ids': tensor([[    0,   113,  1039,  ...,     1,     1,     1],
        [    0,  1106,    47,  ...,     1,     1,     1],
        [    0, 32187, 11824,  ...,     1,     1,     1],
        ...,
        [    0, 31653,  2650,  ...,     1,     1,     1],
        [    0,  7942, 19876,  ...,     1,     1,     1],
        [    0, 14721,  3983,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
{'label': tensor([ 0,  6,  3,  1,  7,  0,  8, 11,  8,  9, 12, 14, 18,  0,  6,  1]), 'input_ids': tensor([[    0,  1039, 12105,  ...,     1,     1,     1],
        [    0, 23029,  2462,  ...,     1,     1,     1],
        [    0,  2387,   235,  ...,     1,     1,     1],
        ...,
        [    0, 104

In [ ]:
from tqdm import tqdm
num_epochs = 1
for epoch in range(num_epochs):
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        # get the inputs
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = emoji_model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        # print running loss for each batch
        running_loss += loss.item()
        avg_loss = running_loss / len(train_loader)
        avg_acc = correct_predictions / total_predictions
        tqdm.write(f'Train Loss: {avg_loss:.3f}, Train Acc: {avg_acc:.3f}', end='\r')
    tqdm.write(f'Epoch {epoch+1}, Train Loss: {avg_loss:.3f}, Train Acc: {avg_acc:.3f}')

    print(f"Epoch {epoch+1} finished")


Epoch 1/1:   0%|          | 1/2813 [00:35<27:50:04, 35.63s/it]

Epoch 1/1:   0%|          | 2/2813 [01:12<28:27:55, 36.46s/it]

Epoch 1/1:   0%|          | 3/2813 [01:47<27:53:04, 35.72s/it]

Epoch 1/1:   0%|          | 4/2813 [02:21<27:27:55, 35.20s/it]

Epoch 1/1:   0%|          | 5/2813 [02:56<27:17:24, 34.99s/it]

...

Epoch 1/1:  83%|████████▎ | 2331/2813 [16:20:55<4:40:23, 34.90s/it]

Epoch 1/1:  83%|████████▎ | 2332/2813 [16:21:34<4:50:25, 36.23s/it]

Epoch 1/1:  83%|████████▎ | 2333/2813 [16:22:11<4:50:36, 36.33s/it]

Epoch 1/1:  83%|████████▎ | 2334/2813 [16:22:57<5:14:51, 39.44s/it]

Epoch 1/1:  83%|████████▎ | 2335/2813 [16:23:30<4:58:56, 37.52s/it]

In [ ]:
# Validation loop
with torch.no_grad():
    emoji_model.eval()  # Set the model to evaluation mode
    valid_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for batch in val_loader:
        # get the inputs
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # forward
        outputs = emoji_model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)

        # calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        # calculate running loss
        valid_loss += loss.item()

    avg_loss = valid_loss / len(val_loader)
    avg_acc = correct_predictions / total_predictions
    print(f'Validation Loss: {avg_loss:.3f}, Validation Acc: {avg_acc:.3f}')

In [ ]:

# Test loop
with torch.no_grad():
    emoji_model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct_predictions = 0
    total_predictions = 0
    for batch in test_loader:
        # get the inputs
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # forward
        outputs = emoji_model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)

        # calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        # calculate running loss
        test_loss += loss.item()

    avg_loss = test_loss / len(test_loader)
    avg_acc = correct_predictions / total_predictions
    print(f'Test Loss: {avg_loss:.3f}, Test Acc: {avg_acc:.3f}')


In [ ]:
# save the model
torch.save(emoji_model.state_dict(), 'Models/emoji_model.pth')
